In [ ]:
# %cd /content/drive/MyDrive
# !git clone https://github.com/dbolya/yolact.git
# %cd yolact

/content/drive/MyDrive
Cloning into 'yolact'...
remote: Enumerating objects: 2936, done.
remote: Total 2936 (delta 0), reused 0 (delta 0), pack-reused 2936
Receiving objects: 100% (2936/2936), 21.20 MiB | 14.26 MiB/s, done.
Resolving deltas: 100% (2002/2002), done.
/content/drive/MyDrive/yolact


GPU 사용 설정 해놓고 하기 / CUDA 안쓰면 오류남

In [ ]:
# install cuda 10.2
!wget https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64/cuda-ubuntu1804.pin
!sudo mv cuda-ubuntu1804.pin /etc/apt/preferences.d/cuda-repository-pin-600
!sudo apt-key adv --fetch-keys https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64/7fa2af80.pub
!sudo add-apt-repository "deb https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64/ /"
!apt-get update
!apt-get -y install cuda-10.2

In [2]:
# install pytorch 1.11
!pip install torch==1.11.0+cu102 torchvision==0.12.0+cu102 torchaudio==0.11.0 --extra-index-url https://download.pytorch.org/whl/cu102

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/, https://download.pytorch.org/whl/cu102
     |████████████████████████████████| 750.6 MB 18 kB/s 
     |████████████████████████████████| 21.0 MB 10.3 MB/s 
     |████████████████████████████████| 2.9 MB 44.1 MB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.12.1+cu113
    Uninstalling torch-1.12.1+cu113:
      Successfully uninstalled torch-1.12.1+cu113
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.13.1+cu113
    Uninstalling torchvision-0.13.1+cu113:
      Successfully uninstalled torchvision-0.13.1+cu113
  Attempting uninstall: torchaudio
    Found existing installation: torchaudio 0.12.1+cu113
    Uninstalling torchaudio-0.12.1+cu113:
      Successfully uninstalled torchaudio-0.12.1+cu113
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source

In [3]:
!nvcc -V

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2019 NVIDIA Corporation
Built on Wed_Oct_23_19:24:38_PDT_2019
Cuda compilation tools, release 10.2, V10.2.89


In [2]:
import torch
print("Torch version:{}".format(torch.__version__))
print("cuda version: {}".format(torch.version.cuda))
print("cudnn version:{}".format(torch.backends.cudnn.version()))

Torch version:1.11.0+cu102
cuda version: 10.2
cudnn version:7605


먼저 압축파일 풀기

In [4]:
#압축파일 풀기
%cd /content
!mkdir Seg
%cd /content/Seg
!unzip -qq "/content/drive/MyDrive/Polygonbox1.zip"

/content
/content/Seg


xml에서 json 뽑기

In [5]:
from glob import glob

folder_path = glob('/content/Seg/*')
print(len(folder_path))

zip_path = glob('/content/Seg/*.zip')

folder_path = [path for path in folder_path if path not in zip_path]

from os import listdir

def fileids(path, ext = None):
    fileList = list()
    path = path if path[-1] == '/' else path + '/'
    for fileName in listdir(path):
        if fileName.endswith(ext):
            fileList.append(path + fileName)
    return fileList

jpg_path = [fileids(path, ext = 'jpg') for path in folder_path]
xml_path = [fileids(path, ext = 'xml') for path in folder_path]
png_path = [fileids(path, ext = 'png') for path in folder_path]
img_path = [jpg + png for jpg,png in zip(jpg_path, png_path)]

print(len(xml_path))
print(len(img_path))

1
1
1


In [6]:
len(img_path[0])

272

In [9]:
%cd /content

/content


In [10]:
from bs4 import BeautifulSoup
from tqdm import tqdm
import json
import numpy as np

#filepath 코드로 xml_path 뽑아서 진행
#coco에 맞게 json 만들기

coco = dict()
images = []
annotations = []
idcount = 1
labeldict = dict()
labelcount = 1
imwanno = []
annoidcount = 1
catids = []

for xml in tqdm(xml_path):
    path = xml[0].replace(xml[0].split('/')[-1], '')
    xml = BeautifulSoup(open(xml[0], encoding='utf-8'),'xml')

    # 이론상 라벨 있어야 할것들
    # for label in xml.select('label'):
    #     lbname = label.text.strip()
    #     if lbname not in labeldict:
    #         labeldict[lbname] = labelcount
    #         labelcount += 1

    for im in xml.select('image'):
        # filename = path + im['name']
        filename = im['name']
        height = int(im['height'])
        width = int(im['width'])
        # id = idcount + im['id'] - 1
        id = idcount
        images.append(
            {
             'id':id,
             'width': width,
             'height': height,
             'file_name': filename
            }
        )

        idcount += 1
        for polygon in im.select('polygon'):
            annoid = annoidcount
            label = polygon['label']
            # 세그멘테이션 데이터셋은 attr이 없음
            # attr = polygon.text.strip('\n')
            # objid = labeldict[label + '_' + attr]

            if label not in labeldict:
                labeldict[label] = labelcount
                labelcount +=1

            objid = labeldict[label]
            segmentation = list(np.array([p.split(',') for p in polygon['points'].split(";")]).flatten().astype(float))
            #area, bbox 구하기
            x = []
            y = []
            for i, f in enumerate(segmentation):
                if i%2==0:
                    x.append(f)
                else:
                    y.append(f)
            area = 0.5*np.abs(np.dot(x,np.roll(y,1))-np.dot(y,np.roll(x,1)))
            bbox = [np.min(x), np.min(y), np.max(x)-np.min(x), np.max(y)-np.min(y)]
            annotations.append(
                {
                 'id': annoid,
                 'image_id': id,
                 'category_id': objid,
                 'segmentation': [segmentation],
                 'area': area,
                 'bbox': bbox,
                 'iscrowd': 0
                }
            )
            annoidcount += 1
            imwanno.append(id)
            catids.append(objid)
    
    # idcount += int(xml.select_one('size').text)
imwanno = list(set(imwanno))
catids = list(set(catids))

coco = {
    'images': images,
    'annotations' : annotations
}        
print(' ')
print('이미지 개수:', len(images))
print('사물 개수:', len(annotations))
print('사물 있는 이미지 개수:',len(imwanno))
print('사물 종류 개수:', len(catids))
print('saving json...')
json_object = json.dumps(coco)
with open('./coco.json', 'w', encoding='utf-8') as make_file:  
    make_file.write(json_object)
print('json saved!')

100%|██████████| 1/1 [00:00<00:00,  4.23it/s]

 
이미지 개수: 272
사물 개수: 1100
사물 있는 이미지 개수: 272
사물 종류 개수: 22
saving json...
json saved!


In [11]:
classname = list()
for k, v in enumerate(labeldict):
    if k+1 in catids:
        classname.append(v)

classname

['pole',
 'person',
 'movable_signage',
 'chair',
 'motorcycle',
 'potted_plant',
 'tree_trunk',
 'car',
 'bicycle',
 'bollard',
 'truck',
 'carrier',
 'barricade',
 'stroller',
 'fire_hydrant',
 'table',
 'traffic_sign',
 'traffic_light',
 'traffic_light_controller',
 'kiosk',
 'scooter',
 'bus']

In [12]:
labeldict

{'pole': 1,
 'person': 2,
 'movable_signage': 3,
 'chair': 4,
 'motorcycle': 5,
 'potted_plant': 6,
 'tree_trunk': 7,
 'car': 8,
 'bicycle': 9,
 'bollard': 10,
 'truck': 11,
 'carrier': 12,
 'barricade': 13,
 'stroller': 14,
 'fire_hydrant': 15,
 'table': 16,
 'traffic_sign': 17,
 'traffic_light': 18,
 'traffic_light_controller': 19,
 'kiosk': 20,
 'scooter': 21,
 'bus': 22}

Yolact train

In [13]:
%cd /content/drive/MyDrive/yolact

/content/drive/MyDrive/yolact


In [14]:
!export CUDA_VISIBLE_DEVICES=[gpus]

In [ ]:
!python train.py --config=yolact_resnet50_config

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!
loading annotations into memory...
Done (t=0.02s)
creating index...
index created!
/usr/local/lib/python3.8/dist-packages/torch/jit/_recursive.py:234: UserWarning: 'lat_layers' was found in ScriptModule constants,  but it is a non-constant submodule. Consider removing it.
  warnings.warn("'{}' was found in ScriptModule constants, "
/usr/local/lib/python3.8/dist-packages/torch/jit/_recursive.py:234: UserWarning: 'downsample_layers' was found in ScriptModule constants,  but it is a non-constant submodule. Consider removing it.
  warnings.warn("'{}' was found in ScriptModule constants, "
/usr/local/lib/python3.8/dist-packages/torch/jit/_recursive.py:234: UserWarning: 'pred_layers' was found in ScriptModule constants,  but it is a non-constant submodule. Consider removing it.
  warnings.warn("'{}' was found in ScriptModule constants, "
Initializing weights...
Begin training!

/content/drive/MyDrive/yolact/ut